In [1]:
!git clone https://github.com/DIKLA2322/QG_demo.git

Cloning into 'QG_demo'...


## Init_QG

In [2]:
!pip install -U transformers==3.0.0

^C


In [ ]:
!python -m nltk.downloader punkt

In [ ]:
!apt install htop

## Init_TC

In [ ]:
!pip install -U torch==1.8.0 torchtext==0.9.0

In [ ]:
!python -m nltk.downloader stopwords

In [ ]:
!apt install htop

## Update

In [ ]:
cd QG_demo/

In [ ]:
!git fetch --all
!git reset --hard origin/master
!git pull

## Text classification

In [ ]:
cd text classification

### Google Word2vec

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
import pprint

In [ ]:
from nltk.corpus import stopwords

# 去停用词
def get_stop_words():
  from nltk.corpus import stopwords
  stop_words = stopwords.words('english')
  for w in ['!',',','.','?','-s','-ly','','s']:
    stop_words.append(w)
  return stop_words

stop_words = get_stop_words()  # 加载停用词表
print(stop_words)

In [ ]:
from nltk import word_tokenize, sent_tokenize
# Create a set of frequent words
stoplist = stop_words

text_corpus = open('./data/text.tsv', 'r', encoding="ISO-8859-1", errors='ignore').read()
# print(text_corpus)
tokens_list = []
processed_corpus = []
text_corpus = text_corpus.split("\n")
# print(text_corpus)
for line in text_corpus:
  # print(text_corpus)
  tokens = word_tokenize(line)   #分词
  tokens = [word.lower() for word in tokens]
  # print(tokens)
  tokens_list.append(tokens)
# print('【NLTK分词结果：】')
print(tokens_list)

interpunctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']   #定义标点符号列表
for tokens in tokens_list:
  processed_tokens = [word for word in tokens if word not in interpunctuations]   #去除标点符号
  filtered_words = [word for word in processed_tokens if word not in stoplist]
# print('\n【NLTK分词后去除符号结果：】')
  processed_corpus.append(filtered_words)
print(processed_corpus)

In [ ]:
from gensim import corpora

dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

In [ ]:
pprint.pprint(dictionary.token2id)

In [ ]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
pprint.pprint(bow_corpus)

In [ ]:
from gensim import models

# train the model
tfidf = models.TfidfModel(bow_corpus)

# transform the "system minors" string
words = "system minors".lower().split()
print(tfidf[dictionary.doc2bow(words)])

In [ ]:
from gensim import similarities

index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=12)

In [ ]:
# query_document = 'system engineering'.split()
# query_bow = dictionary.doc2bow(query_document)
# sims = index[tfidf[query_bow]]
# print(list(enumerate(sims)))

In [ ]:
# for document_number, score in sorted(enumerate(sims), key=lambda x: x[1], reverse=True):
#     print(document_number, score)

In [ ]:
from gensim.models import Word2Vec
# sentences只需要是一个可迭代对象就可以
model = Word2Vec(processed_corpus, min_count=1)  # 执行这一句的时候就是在训练模型了
model.wv.save_word2vec_format('./question.vector')
print(model)

In [ ]:
for i, word in enumerate(model.wv.vocab):
    print(word)

In [ ]:
pairs = [
    ('human', 'production'),   # a minivan is a kind of car
    ('human', 'inference'),   # still a wheeled vehicle
    ('human', 'representative'),  # ok, no wheels, but still a vehicle
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2,model.wv.similarity(w1, w2)))

In [ ]:
!pip install -U torch==1.8.0 torchtext==0.9.0

In [ ]:
import torch
import torchtext
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import jieba
import os
from torch.nn import init
from torchtext.legacy import data
from torchtext.vocab import Vectors
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### data processing


In [ ]:
# 分词
def tokenizer(text): 
    return [word for word in text if word not in stop_words]

In [ ]:
text = data.Field(sequential=True,
                  lower=True,
                  tokenize=tokenizer,
                  stop_words=stop_words)
label = data.Field(sequential=False)
print(text)

In [ ]:
train, val = data.TabularDataset.splits(
    path='data/',
    skip_header=True,
    train='train.tsv',
    validation='validation.tsv',
    format='tsv',
    fields=[('text', text), ('label', label)],
    )

In [ ]:
# print(train[2].text)
# print(train[5].__dict__.keys())


In [ ]:
#加载Google训练的词向量
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('./question.vector', binary=False)
print(model)
print(text)

In [ ]:
cache = 'data/.vector_cache'
if not os.path.exists(cache):
    os.mkdir(cache)
vectors = Vectors(name='./question.vector', cache=cache)
# 指定Vector缺失值的初始化方式，没有命中的token的初始化方式
# vectors.unk_init = nn.init.xavier_uniform_
text.build_vocab(train, val, vectors=vectors)#加入测试集的vertor
print(text)

In [ ]:
text.build_vocab(train, val, vectors=Vectors(name='./question.vector'))#加入测试集的vertor
label.build_vocab(train, val)

embedding_dim = text.vocab.vectors.size()[-1]
vectors = text.vocab.vectors

In [ ]:
text.vocab.freqs.most_common(10)
print(text.vocab.vectors.shape)

In [ ]:

batch_size=128
train_iter, val_iter = data.Iterator.splits(
            (train, val),
            sort_key=lambda x: len(x.text),
            batch_sizes=(batch_size, len(val)), # 训练集设置batch_size,验证集整个集合用于测试
    )

vocab_size = len(text.vocab)
label_num = len(label.vocab)

### model

In [ ]:
class BiLSTM_Attention(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_hiddens, num_layers):
        super(BiLSTM_Attention, self).__init__()
        # embedding之后的shape: torch.Size([200, 8, 300])
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.word_embeddings = self.word_embeddings.from_pretrained(
            vectors, freeze=False)
        # bidirectional设为True即得到双向循环神经网络
        self.encoder = nn.LSTM(input_size=embedding_dim,
                               hidden_size=num_hiddens,
                               num_layers=num_layers,
                               batch_first=False,
                               bidirectional=True)
        # 初始时间步和最终时间步的隐藏状态作为全连接层输入
        self.w_omega = nn.Parameter(torch.Tensor(
            num_hiddens * 2, num_hiddens * 2))
        self.u_omega = nn.Parameter(torch.Tensor(num_hiddens * 2, 1))
        self.decoder = nn.Linear(2*num_hiddens, 2)

        nn.init.uniform_(self.w_omega, -0.1, 0.1)
        nn.init.uniform_(self.u_omega, -0.1, 0.1)

    def forward(self, inputs):
        # print(inputs)
        # inputs的形状是(seq_len,batch_size)
        embeddings = self.word_embeddings(inputs)
        # 提取词特征，输出形状为(seq_len,batch_size,embedding_dim)
        # rnn.LSTM只返回最后一层的隐藏层在各时间步的隐藏状态。
        outputs, _ = self.encoder(embeddings)  # output, (h, c)
        # outputs形状是(seq_len,batch_size, 2 * num_hiddens)
        x = outputs.permute(1, 0, 2)
        # x形状是(batch_size, seq_len, 2 * num_hiddens)
        
        # Attention过程
        u = torch.tanh(torch.matmul(x, self.w_omega))
       # u形状是(batch_size, seq_len, 2 * num_hiddens)
        att = torch.matmul(u, self.u_omega)
       # att形状是(batch_size, seq_len, 1)
        att_score = F.softmax(att, dim=1)
       # att_score形状仍为(batch_size, seq_len, 1)
        scored_x = x * att_score
       # scored_x形状是(batch_size, seq_len, 2 * num_hiddens)
        # Attention过程结束
        
        feat = torch.sum(scored_x, dim=1)
       # feat形状是(batch_size, 2 * num_hiddens)
        outs = self.decoder(feat)
       # out形状是(batch_size, 2)
        return outs

In [ ]:

embedding_dim, num_hiddens, num_layers = 100, 64, 10
net = BiLSTM_Attention(vocab_size, embedding_dim, num_hiddens, num_layers)
print(net)


### train

In [ ]:
def evaluate_accuracy(data_iter,net):
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_iter):
            X, y = batch.text, batch.label
         #   X = X.permute(1, 0)
            y.data.sub_(1)  #X转置 y为啥要减1
            if isinstance(net, torch.nn.Module):
                net.eval() # 评估模式, 这会关闭dropout
                acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
                net.train() # 改回训练模式
            else: # 自定义的模型, 3.13节之后不会用到, 不考虑GPU
                if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                    # 将is_training设置成False
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
            n += y.shape[0]
    return acc_sum / n

In [ ]:

def train(train_iter, test_iter, net, loss, optimizer, num_epochs):
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for batch_idx, batch in enumerate(train_iter):
            X, y = batch.text, batch.label
           # X = X.permute(1, 0)
            y.data.sub_(1)  #X转置 y为啥要减1
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print(
            'epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
            % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n,
               test_acc, time.time() - start))

In [ ]:

lr, num_epochs = 0.01, 10
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss()
train(train_iter, val_iter, net, loss, optimizer, num_epochs)

In [ ]:
net(torch.tensor([[00, 2, 3, 4, 5, 6, 7, 8, 9, 10]])).argmax(dim=1)

## Single task QG


In [ ]:
cd question_generation

In [ ]:
!ls

In [ ]:
from pipelines import pipeline

In [ ]:
nlp = pipeline("question-generation", model="valhalla/t5-base-qg-hl")

In [ ]:
import glob

path_files = glob.glob('process/x_others/*.txt')
dirt = "x_others"
total_file_number = len(path_files)
print("total_file_number: " + str(total_file_number))
begin = 0
for i in range(begin, len(path_files)):
  print("NOW:" + str(i+1) + " / " + str(total_file_number))
  nlp(dirt, "x_others_" + str(i) + ".txt")

In [ ]:
# import glob

# path_files = glob.glob('process/0_prevention_and_control/*.txt')
# dirt = "0_prevention_and_control"
# total_file_number = len(path_files)
# print("total_file_number: " + str(total_file_number))
# begin = 0
# for i in range(begin, len(path_files)):
#   print("NOW:" + str(i+1) + " / " + str(total_file_number))
#   nlp(dirt, "0_prevention_and_control_" + str(i) + ".txt")

In [ ]:
# import glob

# path_files = glob.glob('process/1_health_and_medicine/*.txt')
# dirt = "1_health_and_medicine"
# total_file_number = len(path_files)
# print("total_file_number: " + str(total_file_number))
# begin = 0
# for i in range(begin, len(path_files)):
#   print("NOW:" + str(i+1) + " / " + str(total_file_number))
#   nlp(dirt, "1_health_and_medicine_" + str(i) + ".txt")

In [ ]:
# import glob

# path_files = glob.glob('process/2_event_and_timeline/*.txt')
# dirt = "2_event_and_timeline"
# total_file_number = len(path_files)
# print("total_file_number: " + str(total_file_number))
# begin = 0
# for i in range(begin, len(path_files)):
#   print("NOW:" + str(i+1) + " / " + str(total_file_number))
#   nlp(dirt, "2_event_and_timeline_" + str(i) + ".txt")

In [ ]:
# import glob

# path_files = glob.glob('process/3_character/*.txt')
# dirt = "3_character"
# total_file_number = len(path_files)
# print("total_file_number: " + str(total_file_number))
# begin = 0
# for i in range(begin, len(path_files)):
#   print("NOW:" + str(i+1) + " / " + str(total_file_number))
#   nlp(dirt, "3_character_" + str(i) + ".txt")

In [ ]:
# import glob

# path_files = glob.glob('process/4_epidemic/*.txt')
# dirt = "4_epidemic"
# total_file_number = len(path_files)
# print("total_file_number: " + str(total_file_number))
# begin = 0
# for i in range(begin, len(path_files)):
#   print("NOW:" + str(i+1) + " / " + str(total_file_number))
#   nlp(dirt, "4_epidemic_" + str(i) + ".txt")